In [2]:
import numpy as np
import csv
import numpy
from scipy import stats
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

In [3]:
seed = 5
numpy.random.seed(seed)

# Read the CSV into a pandas data frame (df)
dftr = pd.read_csv('train.csv', delimiter=',')
train_data = np.array(dftr)
dfte = pd.read_csv('test.csv', delimiter=',')
test_data = np.array(dfte)

In [4]:
#Detecting and Removing outliers
train_data = np.array(train_data[:,1:(len(train_data[0]))], dtype=np.float)
#uncomment to see different z score outlier removal results
# z2_traindata = train_data[(np.abs(stats.zscore(train_data)) < 2).all(axis=1)]
# print("Z<2:",len(z2_traindata), "Class 1:",np.count_nonzero(z2_traindata[:,[0]] == 1))

# z25_traindata = train_data[(np.abs(stats.zscore(train_data)) < 2.5).all(axis=1)]
# print("Z<2.5:",len(z25_traindata), "Class 1:",np.count_nonzero(z25_traindata[:,[0]] == 1))

# z3_traindata = train_data[(np.abs(stats.zscore(train_data)) < 3).all(axis=1)]
# print("Z<3:",len(z3_traindata), "Class 1:",np.count_nonzero(z3_traindata[:,[0]] == 1))

# z4_traindata = train_data[(np.abs(stats.zscore(train_data)) < 4).all(axis=1)]
# print("Z<4:",len(z4_traindata), "Class 1:",np.count_nonzero(z4_traindata[:,[0]] == 1))

# z5_traindata = train_data[(np.abs(stats.zscore(train_data)) < 5).all(axis=1)]
# print("Z<5:",len(z5_traindata), "Class 1:",np.count_nonzero(z5_traindata[:,[0]] == 1))

In [5]:
train_data = train_data[(np.abs(stats.zscore(train_data)) < 4).all(axis=1)]
#read csv files to arrays and convert types
XX = train_data[:,0:(len(train_data[0]))]
X = np.array(XX, dtype=np.float)

In [6]:
from spn.algorithms.LearningWrappers import learn_parametric, learn_classifier
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian
from spn.structure.Base import Context

In [7]:
t = [Categorical]
for i in range(200):
	t.append(Gaussian)

#Learning on train data
spn_classification = learn_classifier(X,
                       Context(parametric_types=t).add_domains(X),
                       learn_parametric, 0)


TT = test_data[:,1:(len(test_data[0]))]
R = test_data[:,[0]]

T = np.array(TT, dtype=np.float)
nan = np.array([[np.nan]]*200000)
T = np.append(nan,T,axis=1)
test_classification = T

/home/saber/anaconda3/envs/env-gensim/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saber/anaconda3/envs/env-gensim/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saber/anaconda3/envs/env-gensim/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saber/anaconda3/envs/env-gensim/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saber/anaconda3/envs/env-gensim/lib/python3.6/site-packages/sklearn/cross_decomposition/pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
/home/saber/anaconda3/envs/env-gensim/lib/python3.6/site-packages/sklearn/c

In [8]:
from spn.algorithms.MPE import mpe
#print(mpe(spn_classification, test_classification))		
res = mpe(spn_classification, test_classification)

r = res[:,[0]]
r = np.array(r,dtype=int)
xx = np.append(R,r,axis=1)
np.count_nonzero(r == 1)	
np.savetxt("spn-run-z4.csv", (xx),"%s,%i",header="ID_code,target")

In [10]:
from spn.io.Graphics import plot_spn

plot_spn(spn_classification, 'basicspn.png')

OSError: [Errno 12] Cannot allocate memory

<Figure size 432x288 with 0 Axes>

In [11]:
type(spn_classification)

spn.structure.Base.Sum

In [14]:
from spn.io.Text import spn_to_str_equation
txt = spn_to_str_equation(spn_classification)
with open("spn-dump-z4.txt", "w") as text_file:
    text_file.write(txt)